In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data  import Dataset
from torch.utils.data  import DataLoader
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
%matplotlib inline
device = torch.device('cpu')

In [47]:
#DATASET 설정하기..

In [52]:
class_num=2 #분류 클래스 숫자
scaling_factor=(1.2,1.1,1.15) #scale 집어넣기;;

class DepthwiseConv(nn.Conv2d):
    def __init__(self, in_channels, kernel_size):
        super.__init__(in_channels=in_channels,out_channels=in_channels,kernel_size=kernel_size,groups=in_channels,padding=kernel_size//2)

class SEBlock(nn.Module):
    def __init__(self,channel,reduction_rate):
        super.__init__()
        self.squeeze=AdaptiveAvgPool2d((1,1))#Global Average Pooling
        self.excitation=nn.Sequential(
            nn.Linear(channel,channel//reduction_rate),
            nn.ReLU(),
            nn.Linear(channel//reduction_rate,channel),
            nn.Sigmoid()
        )
        
    def forward(self,x):#x.shpae=(N,C,H,W)
        out=self.squeeze(out)#out.shpae=(N,C,1,1)
        out=out.view(x.size(0),-1)#out.shpae=(N,C)
        out=self.excitation(out)#out.shpae=(N,C)
        out=out.view(x.size(0),x.size(1),1,1)#out.shpae=(N,C,1,1)
        return out

class MBConv1(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1):
        super.__init__()
        self.convi=nn.Sequential(
            nn.Conv2d(in_channels=in_channels,out_channels=in_channels,kernel_size=1),
            nn.BatchNorm2d(),
            nn.ReLU()
        )
        self.DWConv=nn.Sequential(
            DepthwiseConv(in_channels=in_channels,kernel_size=kernel_size),
            nn.BatchNorm2d(),
            nn.ReLU()
        )
        self.SE=SEBlock(channel=in_channels,reduction_rate=8)
        self.convf=nn.Sequential(
            nn.conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=1),
            nn.BatchNorm2d()
        )
            
    def forward(self, x):
        out=self.convi(x)
        out=self.DWConv(out)
        out=self.SE(out)*out
        if stride==1 and in_channels==out_channels:
            out=self.convf(out)+x
        return out

class MBConv6(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1):
        super.__init__()
        self.convi=nn.Sequential(
            nn.Conv2d(in_channels=in_channels,out_channels=6*in_channel,kernel_size=1),
            nn.BatchNorm2d(),
            nn.ReLU()
        )
        self.DWConv=nn.Sequential(
            DepthwiseConv(in_channels=6*in_channel,kernel_size=kernel_size),
            nn.BatchNorm2d(),
            nn.ReLU()
        )
        self.SE=SEBlock(channel=6*in_channels,reduction_rate=8)
        self.convf=nn.Sequential(
            nn.conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=1),
            nn.BatchNorm2d()
        )
            
    def forward(self, x):
        out=self.convi(x)
        out=self.DWConv(out)
        out=self.SE(out)*out
        if stride==1 and in_channels==out_channels:
            out=self.convf(out)+x
        return out

def stage(in_channels,out_channels,kernel_size,default_size, scale):
    net=[]
    stride=1
    s_depth,s_width,_=scale
    for _ in range(int(s_depth*default_size-1)):
        net.append(MBConv6(in_channels=int(s_width*in_channels), out_channels=int(s_width*out_channels),kernel_size=kernel_size,stride=1))
        in_channels=out_channels
    net.append(MBConv6(in_channels=in_channels, out_channels=out_channels,kernel_size=kernel_size,stride=2))
    return nn.Sequential(*net)

class EfficientNetBN(nn.Module):
    def __init__(self, n, class_num, scale):
        super.__init__()
        scale=scale**n
        s_depth,s_width,s_res=scale
        self.stage1=nn.Conv2d(in_channels=int(3*s_width), out_channels=int(32*s_width), kernel_size=3, stride=2)
        self.stage2=MBConv1(in_channels=int(32*s_width), out_channels=int(16*s_width), kernel_size=3)
        self.stage3=stage(16,24,3,2,scale)
        self.stage4=stage(24,40,5,2,scale)
        self.stage5=stage(40,80,3,3,scale)
        self.stage6=stage(80,112,5,3,scale)
        self.stage7=stage(112,192,5,4,scale)
        self.stage8=stage(192,320,5,1,scale)
        self.stage9=nn.Conv2d(in_channels=int(s_width*320),out_channels=int(s_width*1280),kernel_size=1)
        self.pool=nn.AdaptiveAvgPool2d((1,1))
        self.output=nn.Linear(int(s_width*1280),class_num)
    
    def forward(self, x):
        out=self.stage1(x)
        out=self.stage2(out)
        out=self.stage3(out)
        out=self.stage4(out)
        out=self.stage5(out)
        out=self.stage6(out)
        out=self.stage7(out)
        out=self.stage8(out)
        out=self.stage9(out)
        out=self.pool(out)
        out=out.view(x.size(0),-1)#FC layer
        out=out.output(out)
        return out

class EfficientNetB0(nn.Module):
    def __init__(self):
        super.__init__()
        self.stage1=nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3,stride=2)
        self.stage2=MBConv1(in_channels=32, out_channels=16,kernel_size=3)
        self.stage3=nn.Sequential(
            MBConv6(in_channels=16, out_channels=24,kernel_size=3),
            MBConv6(in_channels=24, out_channels=24,kernel_size=3,stride=2)
        )
        self.stage4=nn.Sequential(
            MBConv6(in_channels=24, out_channels=40,kernel_size=5),
            MBConv6(in_channels=40, out_channels=40,kernel_size=5,stride=2)
        )
        self.stage5=nn.Sequential(
            MBConv6(in_channels=40, out_channels=80,kernel_size=3),
            MBConv6(in_channels=80, out_channels=80,kernel_size=3),
            MBConv6(in_channels=80, out_channels=80,kernel_size=3,stride=2)
        )
        self.stage6=nn.Sequential(
            MBConv6(in_channels=80, out_channels=112,kernel_size=5),
            MBConv6(in_channels=112, out_channels=112,kernel_size=5),
            MBConv6(in_channels=112, out_channels=112,kernel_size=5,stride=2)
        )
        self.stage7=nn.Sequential(
            MBConv6(in_channels=112, out_channels=192,kernel_size=5),
            MBConv6(in_channels=192, out_channels=192,kernel_size=5),
            MBConv6(in_channels=192, out_channels=192,kernel_size=5),
            MBConv6(in_channels=192, out_channels=192,kernel_size=5)
        )
        self.stage8=MBConv6(in_channels=192, out_channels=320,kernel_size=5,stride=2)
        self.stage9=nn.Conv2d(in_channels=320,out_channels=1280,kernel_size=1)
        self.pool=nn.AdaptiveAvgPool2d((1,1))
        self.output=nn.Linear(1280,class_num)
    
    def forward(self, x):
        out=self.stage1(x)
        out=self.stage2(out)
        out=self.stage3(out)
        out=self.stage4(out)
        out=self.stage5(out)
        out=self.stage6(out)
        out=self.stage7(out)
        out=self.stage8(out)
        out=self.stage9(out)
        out=self.pool(out)
        out=out.view(x.size(0),-1)#FC layer
        out=out.output(out)
        return out
        

tensor([27,  8])